<a href="https://colab.research.google.com/github/akash-yede/Twittorials/blob/master/Predict_clicks_using_Search_Console_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b>Predict search clicks using Search console data</h1>

We're going to predict the number of search clicks on your website URLs from a Google Search results page.This doesnot include paid Google Ads search results.

We will be predicting the search clicks using Search console data. You should install the google-searchconsole library. Once you install it using the following command, follow the instructions to download your credentails.json and client.json files. Upload the files and access your web property.

In [ ]:
!pip install git+https://github.com/joshcarty/google-searchconsole

First, there is some setup to download a client_id.json file our Python code can use to connect securely to Google Search Console.

Activate Search Console API in Compute Engine https://console.cloud.google.com/apis/api/webmasters.googleapis.com/overview?project=&folder=&organizationId= Create New Credentials / Help me choose (Search Console API, Other UI, User data) https://console.cloud.google.com/apis/credentials/wizard?api=iamcredentials.googleapis.com&project= Download client_id.json

In [ ]:
#upload client_id.json and credentials.json files
from google.colab import files

files.upload()

In [ ]:
import searchconsole

account = searchconsole.authenticate(client_config="client_id.json", serialize='credentials.json', flow="console")

In [ ]:
#Insert your domain name below.
webproperty = account['https://www.domain.com/']

In [ ]:
#let's build a pandas dataframe with the search console data
import pandas as pd

def get_search_console_data(webproperty, days=-365):
  if webproperty is not None:
    query = webproperty.query.range(start='today', days=days).dimension('date')
    r = query.get()
    df = pd.DataFrame(r.rows)
    return df

  print("Web property doesn't exist, please select a valid one from this list")
  print(account.webproperties)

  return None

In [ ]:
df = get_search_console_data(webproperty)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         362 non-null    object 
 1   clicks       362 non-null    int64  
 2   impressions  362 non-null    int64  
 3   ctr          362 non-null    float64
 4   position     362 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 14.3+ KB


In [ ]:
df.head()

,date,clicks,impressions,ctr,position
0,2019-10-24,28,2944,0.009511,56.040082
1,2019-10-25,26,2864,0.009078,59.661662
2,2019-10-26,13,2760,0.004710,59.235145
3,2019-10-27,9,3477,0.002588,60.288755
4,2019-10-28,24,2319,0.010349,59.398448


We will be using the Prophet library from Facebook to predict the search clicks for the next 30, 60 and 90 days. The Prophet library needs an input which is a dataframe of two columns: datestamp (ds) and y (clicks). For this, we will keep only two 

In [ ]:
dft = df[['date','clicks']]

In [ ]:
dft = dft.rename(columns={"date":"ds", "clicks":"y"})

In [ ]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ds      362 non-null    object
 1   y       362 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


We import the Prophet library to predict the clicks. We also import the plot_plotly library to visualize the predictions.

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly

In [ ]:
m = Prophet()
m.fit(dft)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
#Predicting clicks for the next 30 days.
future_30 = m.make_future_dataframe(periods=30)
forecast_30 = m.predict(future_30)
#Predicting clicks for the next 60 days.
future_60 = m.make_future_dataframe(periods=60)
forecast_60 = m.predict(future_60)
#Predicting clicks for the next 90 days.
future_90 = m.make_future_dataframe(periods=90)
forecast_90 = m.predict(future_90)

In [ ]:
#Visualizing the prediction for next 30 days.
plot_plotly(m, forecast_30, xlabel='Date', ylabel='Clicks')

In [ ]:
#Visualizing the prediction for next 60 days.
plot_plotly(m, forecast_60, xlabel='Date', ylabel='Clicks')

In [ ]:
#Visualizing the prediction for next 90 days.
plot_plotly(m, forecast_90, xlabel='Date', ylabel='Clicks')